# Text generation using RNN - Character Level

To generate text using RNN, we need a to convert raw text to a supervised learning problem format.

Take, for example, the following corpus:

"Her brother shook his head incredulously"

First we need to divide the data into tabular format containing input (X) and output (y) sequences. In case of a character level model, the X and y will look like this:

|      X     |  Y  |
|------------|-----|
|    Her b   |  r  |
|    er br   |  o  |
|    r bro   |  t  |
|     brot   |  h  |
|    broth   |  e  |
|    .....   |  .  |
|    .....   |  .  |
|    ulous   |  l  |
|    lousl   |  y  |

Note that in the above problem, the sequence length of X is five characters and that of y is one character. Hence, this is a many-to-one architecture. We can, however, change the number of input characters to any number of characters depending on the type of problem.

A model is trained on such data. To generate text, we simply give the model any five characters using which it predicts the next character. Then it appends the predicted character to the input sequence (on the extreme right of the sequence) and discards the first character (character on extreme left of the sequence). Then it predicts again using the new sequence and the cycle continues until a fix number of iterations. An example is shown below:

Seed text: "incre"

|      X                                            |  Y                       |
|---------------------------------------------------|--------------------------|
|                        incre                      |    < predicted char 1 >  |
|               ncre < predicted char 1 >              |    < predicted char 2 >  |
|       cre< predicted char 1 > < predicted char 2 >   |    < predicted char 3 >  |
|       re< predicted char 1 >< predicted char 2 > < predicted char 3 >   |    < predicted char 4 >  |
|                      ...                          |            ...           |

# Notebook Overview
1. Preprocess data
2. LSTM model
3. Generate code

In [1]:
# import libraries
import warnings
warnings.filterwarnings("ignore")

import os
import re
import numpy as np
import random
import sys
import io
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils.data_utils import get_file

Using TensorFlow backend.


# 1. Preprocess data

We're going to build a C code generator by training an RNN on a huge corpus of C code (the linux kernel code). You can download the C code used as source text from the following link:
https://github.com/torvalds/linux/tree/master/kernel

We have already downloaded the entire kernel folder and stored in a local directory

## Load C code

In [2]:
# set path where C files reside

path = r"storage/linux-kernel"

os.chdir(path)

file_names = os.listdir()
print(file_names)

['arraymap.c', 'internals.h', 'trace_kprobe.c', 'swait.c', 'gcc_4_7.c', 'trace_functions_graph.c', 'elfcore.c', 'timer.c', 'ring_buffer.c', 'user-return-notifier.c', 'base.c', 'trace_kprobe_selftest.c', 'proc.c', 'resend.c', 'module-internal.h', 'lockdep_states.h', 'swap.c', 'power.h', 'timekeeping.h', 'clockevents.c', 'jump_label.c', 'console_cmdline.h', 'rstat.c', 'printk_safe.c', 'blktrace.c', 'trace_benchmark.h', 'syscall.c', 'devres.c', 'tick-sched.c', 'kdb_main.c', 'smpboot.h', 'autogroup.h', 'cpu_pm.c', 'cpumap.c', 'utsname_sysctl.c', 'timekeeping_debug.c', 'trace_kdb.c', 'Kconfig.hz', 'trace_event_perf.c', 'cpudeadline.h', 'posix-clock.c', 'pelt.c', 'ring_buffer_benchmark.c', 'trace_nop.c', 'user.c', 'loadavg.c', 'timekeeping.c', 'tick-oneshot.c', 'isolation.c', 'stats.c', 'lockdep_internals.h', 'trace_events_filter_test.h', 'cpuset.c', 'torture.c', 'wait.c', 'trace.h', 'kdb_bt.c', 'chip.c', 'trace_events.c', 'core.h', 'exit.c', 'pid.c', 'offload.c', 'irq_sim.c', 'sched_clock.c

In [3]:
# use regex to filter .c files
import re
c_names = ".*\.c$"

c_files = list()

for file in file_names:
    if re.match(c_names, file):
        c_files.append(file)

print(c_files)

['arraymap.c', 'trace_kprobe.c', 'swait.c', 'gcc_4_7.c', 'trace_functions_graph.c', 'elfcore.c', 'timer.c', 'ring_buffer.c', 'user-return-notifier.c', 'base.c', 'trace_kprobe_selftest.c', 'proc.c', 'resend.c', 'swap.c', 'clockevents.c', 'jump_label.c', 'rstat.c', 'printk_safe.c', 'blktrace.c', 'syscall.c', 'devres.c', 'tick-sched.c', 'kdb_main.c', 'cpu_pm.c', 'cpumap.c', 'utsname_sysctl.c', 'timekeeping_debug.c', 'trace_kdb.c', 'trace_event_perf.c', 'posix-clock.c', 'pelt.c', 'ring_buffer_benchmark.c', 'trace_nop.c', 'user.c', 'loadavg.c', 'timekeeping.c', 'tick-oneshot.c', 'isolation.c', 'stats.c', 'cpuset.c', 'torture.c', 'wait.c', 'kdb_bt.c', 'chip.c', 'trace_events.c', 'exit.c', 'pid.c', 'offload.c', 'irq_sim.c', 'sched_clock.c', 'profile.c', 'kdb_keyboard.c', 'timeconv.c', 'tick-broadcast.c', 'clocksource.c', 'patch.c', 'trace_stat.c', 'xskmap.c', 'handle.c', 'mutex-debug.c', 'reboot.c', 'generic-chip.c', 'taskstats.c', 'stop_machine.c', 'tick-broadcast-hrtimer.c', 'async.c', 'tra

In [4]:
# load all c code in a list
full_code = list()
for file in c_files:
    code = open(file, "r", encoding='utf-8')
    full_code.append(code.read())
    code.close()

In [5]:
# let's look at how a typical C code looks like
print(full_code[20])

// SPDX-License-Identifier: GPL-2.0
#include <linux/module.h>
#include <linux/interrupt.h>
#include <linux/device.h>
#include <linux/gfp.h>
#include <linux/irq.h>

#include "internals.h"

/*
 * Device resource management aware IRQ request/free implementation.
 */
struct irq_devres {
	unsigned int irq;
	void *dev_id;
};

static void devm_irq_release(struct device *dev, void *res)
{
	struct irq_devres *this = res;

	free_irq(this->irq, this->dev_id);
}

static int devm_irq_match(struct device *dev, void *res, void *data)
{
	struct irq_devres *this = res, *match = data;

	return this->irq == match->irq && this->dev_id == match->dev_id;
}

/**
 *	devm_request_threaded_irq - allocate an interrupt line for a managed device
 *	@dev: device to request interrupt for
 *	@irq: Interrupt line to allocate
 *	@handler: Function to be called when the IRQ occurs
 *	@thread_fn: function to be called in a threaded interrupt context. NULL
 *		    for devices which handle everything in @handler
 *	@irqfla

In [6]:
# merge different c codes into one big c code
text = "\n".join(full_code)
print("Total number of characters in entire code: {}".format(len(text)))

Total number of characters in entire code: 6615537


In [7]:
# top_n: only consider first top_n characters and discard the rest for memory and computational efficiency
top_n = 400000
text = text[:top_n]

## Convert characters to integers

In [8]:
# create character to index mapping
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [9]:
print("Vocabulary size: {}".format(len(chars)))

Vocabulary size: 97


## Divide data in input (X) and output (y)

### Create sequences

In [10]:
# define length for each sequence
MAX_SEQ_LENGTH = 50          # number of input characters (X) in each sequence 
STEP           = 3           # increment between each sequence
VOCAB_SIZE     = len(chars)  # total number of unique characters in dataset

sentences  = []              # X
next_chars = []              # y

for i in range(0, len(text) - MAX_SEQ_LENGTH, STEP):
    sentences.append(text[i: i + MAX_SEQ_LENGTH])
    next_chars.append(text[i + MAX_SEQ_LENGTH])

In [11]:
print('Number of training samples: {}'.format(len(sentences)))

Number of training samples: 133317


## Create input and output using the created sequences

When you're not using the Embedding layer of the Keras as the very first layer, you need to convert your data in the following format:
#### input shape should be of the form :  (#samples, #timesteps, #features)
#### output shape should be of the form :  (#samples, #timesteps, #features)

![Tensor shape](./jupyter resources/rnn_tensor.png)

#samples: the number of data points (or sequences)
#timesteps: It's the length of the sequence of your data (the MAX_SEQ_LENGTH variable).
#features: Number of features depends on the type of problem. In this problem, #features is the vocabulary size, that is, the dimensionality of the one-hot encoding matrix using which each character is being represented. If you're working with **images**, features size will be equal to: (height, width, channels), and the input shape will be (#training_samples, #timesteps, height, width, channels)

In [12]:
# create X and y
X = np.zeros((len(sentences), MAX_SEQ_LENGTH, VOCAB_SIZE), dtype=np.bool)
y = np.zeros((len(sentences), VOCAB_SIZE), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [13]:
print("Shape of X: {}".format(X.shape))
print("Shape of y: {}".format(y.shape))

Shape of X: (133317, 50, 97)
Shape of y: (133317, 97)


Here, X is reshaped to (#samples, #timesteps, #features). We have explicitly mentioned the third dimension (#features) because we won't use the Embedding() layer of Keras in this case since there are only 97 characters. Characters can be represented as one-hot encoded vector. There are no word embeddings for characters.

# 2. LSTM

In [17]:
# define model architecture - using a two-layer LSTM with 128 LSTM cells in each layer
model = Sequential()
model.add(LSTM(128, input_shape=(MAX_SEQ_LENGTH, VOCAB_SIZE), return_sequences=True, dropout=1.0))
model.add(LSTM(128, dropout=1.0))
model.add(Dense(VOCAB_SIZE, activation = "softmax"))

optimizer = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ['acc'])

In [18]:
# check model summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 50, 128)           115712    
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 97)                12513     
Total params: 259,809
Trainable params: 259,809
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# fit model
model.fit(X, y, batch_size=128, epochs=20)

Epoch 1/20
133317/133317 [==============================] - 157s 1ms/step - loss: 2.3676 - acc: 0.3783
Epoch 2/20
133317/133317 [==============================] - 156s 1ms/step - loss: 1.6232 - acc: 0.5533
Epoch 3/20
133317/133317 [==============================] - 155s 1ms/step - loss: 1.4276 - acc: 0.5989
Epoch 4/20
133317/133317 [==============================] - 156s 1ms/step - loss: 1.3328 - acc: 0.6212
Epoch 5/20
133317/133317 [==============================] - 157s 1ms/step - loss: 1.3376 - acc: 0.6185
Epoch 6/20
133317/133317 [==============================] - 154s 1ms/step - loss: 1.3244 - acc: 0.6221
Epoch 7/20
133317/133317 [==============================] - 155s 1ms/step - loss: 1.2058 - acc: 0.6524
Epoch 8/20
133317/133317 [==============================] - 156s 1ms/step - loss: 1.1513 - acc: 0.6650
Epoch 9/20
133317/133317 [==============================] - 154s 1ms/step - loss: 1.1187 - acc: 0.6747
Epoch 10/20
133317/133317 [==============================] - 155s 1ms/ste

# 3. Generate code

Create a function that will make next character predictions based on temperature. If temperature is greater than 1, the generated characters will be more versatile and diverse. On the other hand, if temperature is less than one, the generated characters will be much more conservative.

In [35]:
# define function to sample next word from a probability array based on temperature
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [48]:
np.random.multinomial(10, [0.05, 0.9, 0.05], size=2)

array([[1, 9, 0],
       [1, 9, 0]])

In [36]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random code to start text generation

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

-------------------------------------------------- diversity: 0.5
----- Generating with seed: "ed, or inherited, filter
 * @prog: the BPF program"
ed, or inherited, filter
 * @prog: the BPF program and the task and task or the tracin

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


g_map_elt
 * packet threads the table the table in both the RSH the
	 * but state */
		if (cpumask_itec_idag(struct sched_prove_thread_clock_irq_desc(&status_in_buffer, 0;
		if (sd->call = cpu_to_node(struct trace_event_file *file->event_call)
{
	int err < thread_conf[ids <= 0) {
		trace_array *parent;
	if (!struct task_struct *ptrace_thread_canter(cpu);
						if (char *current->seccomp_filter(tr);
	ret = tracing_map_elt;
	struct trace_event_file *file->file->filter_bin_nota(tr);
	if (!call->class->system(olize, restart_show_reg));
	if (!strint_subsystem_compat, int file->event_call)
{
	struct trace_event_file *file;
	char *tsk->spat(map->thread(&trace_thread(struct trace_event_file *file, *map)
{
	int trace_event_enable_mutex);
	if (!exit_idx)
						break;
		break;
	}
	return -ENODLIBL2;
	char *names_compan_type, task, path;
		list_filter_type(parst_cpu_code = 0;
		list_filter(p, &tr->events, cnable_return(struct trace_event_file *file)
{
	unsigned ------------------------------------

In [37]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random seed

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

-------------------------------------------------- diversity: 0.5
----- Generating with seed: "sd)), sd->name);

	if (!cpumask_test_cpu(cpu, sche"
sd)), sd->name);

	if (!cpumask_test_cpu(cpu, sched_domain_

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


balance_table[] = {
	{ CTL_INT,	NET_IPV4_ROUTE_PROFB_TIMEOUT,		"stancs_table },
	{ CTL_INT,	NET_IPV4_ROUTE_THREAD_CPUCLOCK_CHR_CPU),		"do_cache_to_usent_table },
	{ CTL_INT,	NET_IPV4_ROUTE_SCS			"thread_conf_type_topalled_table },
	{ CTL_INT,	NET_NECNEK_DESTR(current)
			return ret;
}

static int cpupri;
	if (!strlen(struct trace_event_file *file)
{
	struct trace_event_file *file;

	if (unlikely(!name;
	char *ppos, struct trace_array *tr)
{
	struct trace_event_file *file->file;
	int proc_countr *sd)
{
	int task_signoot_state(tsk->timer);
	if (!struct task_struct *tsk)
					swaty_cache_seccomp_actions_lock);

	if (edata->compat_task_seccomp_actions_lock);
		if (sd->cpu_to_thread_group_t buf_trace_init(tsk))
																       struct task_struct *tr = event_seccomp.filter);
	char *ptr == 0)
			char *tsk;
	int irqdh-NULL;
			list_filter_pid_msg(don_is_read(rescr, task();
		if (!strlen(file);
		if (tsk->signal->wait_lock);
		if (!file->event_set_call);
	if (!span = trace_threa---------